## MACHINE LEARNING

* LightGBM

In [1]:
import time
import numpy as np
import pandas as pd
import lightgbm as lgb
import math
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

In [2]:
train = pd.read_csv(r'train.csv', parse_dates=['date'])
test = pd.read_csv(r'test.csv', parse_dates=['date'])
df = pd.concat([train, test], sort=False)

In [3]:
def create_date_features(df):
    df['month']=df.date.dt.month
    df['year']=df.date.dt.year
    df['day_of_month']=df.date.dt.day
    df['day_of_year']=df.date.dt.dayofyear
    df['week_of_year']=df.date.dt.weekofyear
    df['day_of_week']=df.date.dt.dayofweek + 1
    df['is_wknd']=df.date.dt.weekday // 4
    df['is_month_start']=df.date.dt.is_month_start.astype(int)
    df['is_month_end']=df.date.dt.is_month_end.astype(int)
    df['is_quarter_end']=df.date.dt.is_quarter_end.astype(int)
    df['is_quarter_start']=df.date.dt.is_quarter_start.astype(int)
    df['is_year_end']=df.date.dt.is_year_end.astype(int)
    df['is_year_start']=df.date.dt.is_year_start.astype(int)
    df['week_in_month'] = pd.to_numeric(df.date.dt.day / 7)
    df['week_in_month'] = df['week_in_month'].apply(lambda x: math.ceil(x))
    df.loc[df['date'].dt.strftime('%m-%d') == '02-14', 'is_14_Feb'] = 1
    df.loc[df['date'].dt.strftime('%m-%d') != '02-14', 'is_14_Feb'] = 0
    df.loc[df['date'].dt.strftime('%m-%d') == '07-04', 'is_04_Jul'] = 1
    df.loc[df['date'].dt.strftime('%m-%d') != '07-04', 'is_04_Jul'] = 0
    blackfri_list = ['2013-11-29','2014-11-28','2015-11-27','2016-11-25','2017-11-24','2018-11-23']
    df.loc[df['date'].isin(blackfri_list),'is_black_Fri'] = 1
    df.loc[~df['date'].isin(blackfri_list),'is_black_Fri'] = 0
    return df

df = create_date_features(df)
df.head()

,date,store,item,sales,id,month,year,day_of_month,day_of_year,week_of_year,day_of_week,is_wknd,is_month_start,is_month_end,is_quarter_end,is_quarter_start,is_year_end,is_year_start,week_in_month,is_14_Feb,is_04_Jul,is_black_Fri
0,2013-01-01,1,1,13.0,NaN,1,2013,1,1,1,2,0,1,0,0,1,0,1,1,0.0,0.0,0.0
1,2013-01-02,1,1,11.0,NaN,1,2013,2,2,1,3,0,0,0,0,0,0,0,1,0.0,0.0,0.0
2,2013-01-03,1,1,14.0,NaN,1,2013,3,3,1,4,0,0,0,0,0,0,0,1,0.0,0.0,0.0
3,2013-01-04,1,1,13.0,NaN,1,2013,4,4,1,5,1,0,0,0,0,0,0,1,0.0,0.0,0.0
4,2013-01-05,1,1,10.0,NaN,1,2013,5,5,1,6,1,0,0,0,0,0,0,1,0.0,0.0,0.0


In [4]:
df.sort_values(by=['store', 'item', 'date'], axis=0, inplace=True)

### Random Noise

In [5]:
def random_noise(dataframe):
    return np.random.normal(scale=1.6, size=(len(dataframe),))

### Lag/Shifted Features

In [6]:
def lag_features(dataframe, lags):
    dataframe = dataframe.copy()
    for lag in lags:
        dataframe['sales_lag_' + str(lag)] = dataframe.groupby(["store", "item"])['sales'].transform(
            lambda x: x.shift(lag)) + random_noise(dataframe)
    return dataframe


df = lag_features(df, [91, 98, 105, 112, 119, 126, 182, 364, 546, 728])
df.head(95)

,date,store,item,sales,id,month,year,day_of_month,day_of_year,week_of_year,day_of_week,is_wknd,is_month_start,is_month_end,is_quarter_end,is_quarter_start,is_year_end,is_year_start,week_in_month,is_14_Feb,is_04_Jul,is_black_Fri,sales_lag_91,sales_lag_98,sales_lag_105,sales_lag_112,sales_lag_119,sales_lag_126,sales_lag_182,sales_lag_364,sales_lag_546,sales_lag_728
0,2013-01-01,1,1,13.0,NaN,1,2013,1,1,1,2,0,1,0,0,1,0,1,1,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-01-02,1,1,11.0,NaN,1,2013,2,2,1,3,0,0,0,0,0,0,0,1,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-01-03,1,1,14.0,NaN,1,2013,3,3,1,4,0,0,0,0,0,0,0,1,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-01-04,1,1,13.0,NaN,1,2013,4,4,1,5,1,0,0,0,0,0,0,1,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-01-05,1,1,10.0,NaN,1,2013,5,5,1,6,1,0,0,0,0,0,0,1,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,2013-04-01,1,1,11.0,NaN,4,2013,1,91,14,1,0,1,0,0,1,0,0,1,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,2013-04-02,1,1,19.0,NaN,4,2013,2,92,14,2,0,0,0,0,0,0,0,1,0.0,0.0,0.0,10.990394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,2013-04-03,1,1,24.0,NaN,4,2013,3,93,14,3,0,0,0,0,0,0,0,1,0.0,0.0,0.0,10.667555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,2013-04-04,1,1,18.0,NaN,4,2013,4,94,14,4,0,0,0,0,0,0,0,1,0.0,0.0,0.0,14.456123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Rolling Mean Features

In [7]:
def roll_mean_features(dataframe, windows):
    dataframe = dataframe.copy()
    for window in windows:
        dataframe['sales_roll_mean_' + str(window)] = dataframe.groupby(["store", "item"])['sales']. \
                                                          transform(
            lambda x: x.shift(1).rolling(window=window, min_periods=10, win_type="triang").mean()) + random_noise(
            dataframe)
    return dataframe


df = roll_mean_features(df, [365, 546])
df.head(15)

,date,store,item,sales,id,month,year,day_of_month,day_of_year,week_of_year,day_of_week,is_wknd,is_month_start,is_month_end,is_quarter_end,is_quarter_start,is_year_end,is_year_start,week_in_month,is_14_Feb,is_04_Jul,is_black_Fri,sales_lag_91,sales_lag_98,sales_lag_105,sales_lag_112,sales_lag_119,sales_lag_126,sales_lag_182,sales_lag_364,sales_lag_546,sales_lag_728,sales_roll_mean_365,sales_roll_mean_546
0,2013-01-01,1,1,13.0,NaN,1,2013,1,1,1,2,0,1,0,0,1,0,1,1,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-01-02,1,1,11.0,NaN,1,2013,2,2,1,3,0,0,0,0,0,0,0,1,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-01-03,1,1,14.0,NaN,1,2013,3,3,1,4,0,0,0,0,0,0,0,1,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-01-04,1,1,13.0,NaN,1,2013,4,4,1,5,1,0,0,0,0,0,0,1,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-01-05,1,1,10.0,NaN,1,2013,5,5,1,6,1,0,0,0,0,0,0,1,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2013-01-06,1,1,12.0,NaN,1,2013,6,6,1,7,1,0,0,0,0,0,0,1,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2013-01-07,1,1,10.0,NaN,1,2013,7,7,2,1,0,0,0,0,0,0,0,1,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2013-01-08,1,1,9.0,NaN,1,2013,8,8,2,2,0,0,0,0,0,0,0,2,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2013-01-09,1,1,12.0,NaN,1,2013,9,9,2,3,0,0,0,0,0,0,0,2,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2013-01-10,1,1,9.0,NaN,1,2013,10,10,2,4,0,0,0,0,0,0,0,2,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Exponentially Weighted Mean Features

In [8]:
def ewm_features(dataframe, alphas, lags):
    dataframe = dataframe.copy()
    for alpha in alphas:
        for lag in lags:
            dataframe['sales_ewm_alpha_' + str(alpha).replace(".", "") + "_lag_" + str(lag)] = \
                dataframe.groupby(["store", "item"])['sales'].transform(lambda x: x.shift(lag).ewm(alpha=alpha).mean())
    return dataframe


alphas = [0.95, 0.9, 0.8, 0.7, 0.5]
lags = [91, 98, 105, 112, 180, 270, 365, 546, 728]

df = ewm_features(df, alphas, lags)
df.head(95)

,date,store,item,sales,id,month,year,day_of_month,day_of_year,week_of_year,day_of_week,is_wknd,is_month_start,is_month_end,is_quarter_end,is_quarter_start,is_year_end,is_year_start,week_in_month,is_14_Feb,is_04_Jul,is_black_Fri,sales_lag_91,sales_lag_98,sales_lag_105,sales_lag_112,sales_lag_119,sales_lag_126,sales_lag_182,sales_lag_364,sales_lag_546,sales_lag_728,sales_roll_mean_365,sales_roll_mean_546,sales_ewm_alpha_095_lag_91,sales_ewm_alpha_095_lag_98,sales_ewm_alpha_095_lag_105,sales_ewm_alpha_095_lag_112,sales_ewm_alpha_095_lag_180,sales_ewm_alpha_095_lag_270,sales_ewm_alpha_095_lag_365,sales_ewm_alpha_095_lag_546,sales_ewm_alpha_095_lag_728,sales_ewm_alpha_09_lag_91,sales_ewm_alpha_09_lag_98,sales_ewm_alpha_09_lag_105,sales_ewm_alpha_09_lag_112,sales_ewm_alpha_09_lag_180,sales_ewm_alpha_09_lag_270,sales_ewm_alpha_09_lag_365,sales_ewm_alpha_09_lag_546,sales_ewm_alpha_09_lag_728,sales_ewm_alpha_08_lag_91,sales_ewm_alpha_08_lag_98,sales_ewm_alpha_08_lag_105,sales_ewm_alpha_08_lag_112,sales_ewm_alpha_08_lag_180,sales_ewm_alpha_08_lag_270,sales_ewm_alpha_08_lag_365,sales_ewm_alpha_08_lag_546,sales_ewm_alpha_08_lag_728,sales_ewm_alpha_07_lag_91,sales_ewm_alpha_07_lag_98,sales_ewm_alpha_07_lag_105,sales_ewm_alpha_07_lag_112,sales_ewm_alpha_07_lag_180,sales_ewm_alpha_07_lag_270,sales_ewm_alpha_07_lag_365,sales_ewm_alpha_07_lag_546,sales_ewm_alpha_07_lag_728,sales_ewm_alpha_05_lag_91,sales_ewm_alpha_05_lag_98,sales_ewm_alpha_05_lag_105,sales_ewm_alpha_05_lag_112,sales_ewm_alpha_05_lag_180,sales_ewm_alpha_05_lag_270,sales_ewm_alpha_05_lag_365,sales_ewm_alpha_05_lag_546,sales_ewm_alpha_05_lag_728
0,2013-01-01,1,1,13.0,NaN,1,2013,1,1,1,2,0,1,0,0,1,0,1,1,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-01-02,1,1,11.0,NaN,1,2013,2,2,1,3,0,0,0,0,0,0,0,1,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-01-03,1,1,14.0,NaN,1,2013,3,3,1,4,0,0,0,0,0,0,0,1,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-01-04,1,1,13.0,NaN,1,2013,4,4,1,5,1,0,0,0,0,0,0,1,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-01-05,1,1,10.0,NaN,1,2013,5,5,1,6,1,0,0,0,0,0,0,1,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,2013-04-01,1,1,11.0,NaN,4,2013,1,91,14,1,0,1,0,0,1,0,0,1,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.313700,10.444661,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,2013-04-02,1,1,19.0,NaN,4,2013,2,92,14,2,0,0,0,0,0,0,0,1,0.0,0.0,0.0,10.990394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.887974,12.887432,13.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

### One Hot Encoding

In [9]:
df = pd.get_dummies(df, columns=['store', 'item', 'day_of_week', 'month'])

### Converting sales to log(1+sales)

In [10]:
df['sales'] = np.log1p(df["sales"].values)

### Custom Cost Function (SMAPE)

In [11]:
def smape(preds, target):
    n = len(preds)
    masked_arr = ~((preds == 0) & (target == 0))
    preds, target = preds[masked_arr], target[masked_arr]
    num = np.abs(preds - target)
    denom = np.abs(preds) + np.abs(target)
    smape_val = (200 * np.sum(num / denom)) / n
    return smape_val


def lgbm_smape(preds, train_data):
    labels = train_data.get_label()
    smape_val = smape(np.expm1(preds), np.expm1(labels))
    return 'SMAPE', smape_val, False

### Time-based Validation Sets

In [12]:
train = df.loc[(df["date"] < "2017-01-01"), :]
val = df.loc[(df["date"] >= "2017-01-01") & (df["date"] < "2017-04-01"), :]
cols = [col for col in train.columns if col not in ['date', 'id', "sales", "year"]]

Y_train = train['sales']
X_train = train[cols]

Y_val = val['sales']
X_val = val[cols]

### LightGBM Model

In [13]:
lgb_params = {'metric': {'mae'},
              'num_leaves': 10,
              'learning_rate': 0.02,
              'feature_fraction': 0.8,
              'max_depth': 5,
              'verbose': 0,
              'num_boost_round': 1000,
              'early_stopping_rounds': 200,
              'nthread': -1}


lgbtrain = lgb.Dataset(data=X_train, label=Y_train, feature_name=cols)
lgbval = lgb.Dataset(data=X_val, label=Y_val, reference=lgbtrain, feature_name=cols)

model = lgb.train(lgb_params, lgbtrain,
                  valid_sets=[lgbtrain, lgbval],
                  num_boost_round=lgb_params['num_boost_round'],
                  early_stopping_rounds=lgb_params['early_stopping_rounds'],
                  feval=lgbm_smape,
                  verbose_eval=100)

y_pred_val = model.predict(X_val, num_iteration=model.best_iteration)
print("Validation SMAPE", smape(np.expm1(y_pred_val), np.expm1(Y_val)))

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.223952 seconds.
You can set `force_col_wise=true` to remove the overhead.
Training until validation scores don't improve for 200 rounds
[100]	training's l1: 0.172354	training's SMAPE: 17.5773	valid_1's l1: 0.171092	valid_1's SMAPE: 17.4874
[200]	training's l1: 0.142141	training's SMAPE: 14.5581	valid_1's l1: 0.144889	valid_1's SMAPE: 14.8637
[300]	training's l1: 0.136589	training's SMAPE: 14.0024	valid_1's l1: 0.140415	valid_1's SMAPE: 14.4149
[400]	training's l1: 0.134489	training's SMAPE: 13.793	valid_1's l1: 0.138693	valid_1's SMAPE: 14.2425
[500]	training's l1: 0.133237	training's SMAPE: 13.6682	valid_1's l1: 0.137474	valid_1's SMAPE: 14.1201
[600]	training's l1: 0.132335	training's SMAPE: 13.5782	valid_1's l1: 0.136534	valid_1's SMAPE: 14.0256
[700]	training's l1: 0.13164	training's SMAPE: 13.5088	valid_1's l1: 0.135886	valid_1's SMAPE: 13.9604
[800]	training's l1: 0.131072	training's SMAPE:

### Feature Importance

In [14]:
def plot_lgb_importances(model, plot=False, num=10):
    from matplotlib import pyplot as plt
    import seaborn as sns
    gain = model.feature_importance('gain')
    feat_imp = pd.DataFrame({'feature': model.feature_name(),
                             'split': model.feature_importance('split'),
                             'gain': 100 * gain / gain.sum()}).sort_values('gain', ascending=False)
    if plot:
        plt.figure(figsize=(10, 10))
        sns.set(font_scale=1)
        sns.barplot(x="gain", y="feature", data=feat_imp[0:25])
        plt.title('feature')
        plt.tight_layout()
        plt.show()
    else:
        print(feat_imp.head(num))


plot_lgb_importances(model, num=30)

                         feature  split       gain
25           sales_roll_mean_546   1009  56.512028
21                 sales_lag_364   1214  12.205399
24           sales_roll_mean_365    650   6.071676
68    sales_ewm_alpha_05_lag_365    348   4.414820
53     sales_ewm_alpha_07_lag_91     29   4.007032
1                    day_of_year    744   2.092183
62     sales_ewm_alpha_05_lag_91     77   1.852987
59    sales_ewm_alpha_07_lag_365     55   1.502267
26    sales_ewm_alpha_095_lag_91     62   1.404985
131                day_of_week_1    236   1.367029
14                  sales_lag_91     79   1.233292
3                        is_wknd    234   1.191631
149                     month_12    310   1.055317
2                   week_of_year    291   0.971130
15                  sales_lag_98     21   0.474175
70    sales_ewm_alpha_05_lag_728    381   0.373951
67    sales_ewm_alpha_05_lag_270    196   0.360551
61    sales_ewm_alpha_07_lag_728     67   0.297357
52    sales_ewm_alpha_08_lag_72

In [15]:
### Final Model

In [16]:
train = df.loc[~df.sales.isna()]
Y_train = train['sales']
X_train = train[cols]

test = df.loc[df.sales.isna()]
X_test = test[cols]

lgb_params = {'metric': {'mae'},
              'num_leaves': 10,
              'learning_rate': 0.02,
              'feature_fraction': 0.8,
              'max_depth': 5,
              'verbose': 0,
              'nthread': -1,
              "num_boost_round": model.best_iteration}

# LightGBM dataset
lgbtrain_all = lgb.Dataset(data=X_train, label=Y_train, feature_name=cols)
final_model = lgb.train(lgb_params, lgbtrain_all, num_boost_round=model.best_iteration)
test_preds = final_model.predict(X_test, num_iteration=model.best_iteration)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.312762 seconds.
You can set `force_col_wise=true` to remove the overhead.
